<a href="https://colab.research.google.com/github/rhks13/DART/blob/main/%EB%8B%A4%EC%A4%91%ED%9A%8C%EA%B7%80%EB%B6%84%EC%84%9D_%ED%8C%A9%ED%84%B0%EB%B3%84%EA%B0%80%EC%A4%91%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np



# 퀄리티 factor를 통한 유니버스 제한

In [ ]:
# 데이터 업로드
total_income = pd.read_excel('C:/Users/82106/\Desktop/DART/코스피_매출총이익.xlsx', index_col=0)
total_assets = pd.read_excel('C:/Users/82106/\Desktop/DART/코스피_총자산.xlsx', index_col=0)
pbr = pd.read_excel('C:/Users/82106/\Desktop/DART/코스피_PBR.xlsx', index_col=0)

In [ ]:
# gpa 생성
gpa=total_income/total_assets
# pbr rank 생성
pbr_rank=pbr.describe().loc["mean"]/pbr.describe().loc["count"]
pbr_rank=pd.DataFrame(pbr_rank)
pbr_rank['rank'] = pbr_rank[0].rank()
# gpa rank 생성
gpa_rank=gpa.describe().loc["mean"]/gpa.describe().loc["count"]
gpa_rank=pd.DataFrame(gpa_rank)
gpa_rank['rank'] = gpa_rank[0].rank()
# quality rank 생성
quality_rank = pbr_rank+gpa_rank
quality_rank['final_rank'] = quality_rank['rank'].rank()
quality_rank=quality_rank.sort_values("final_rank")[394:794]
# 유니버스 제한
quality_index=list(quality_rank.index)

# 상승장세 파악

In [ ]:
kospi = pd.read_excel('C:/Users/82106/\Desktop/DART/코스피_지수.xlsx', index_col=0)
kospi= kospi.diff()
kospi_inc=kospi[kospi>0].dropna(axis=0)

#각행을 한달치의 차분이라 생각.
#ex) 8/31 -> 7월~8월

In [ ]:
kospi_inc

,소계
Symbol Name,
2002-08-31,18.41
2002-10-31,12.50
2002-11-30,65.88
2003-04-30,63.65
2003-05-31,34.07
...,...
2021-05-31,56.06
2021-06-30,92.76
2021-12-31,138.64


In [ ]:
# 상승장세 인덱스 
inc_index=kospi_inc.index
inc_index

DatetimeIndex(['2002-08-31', '2002-10-31', '2002-11-30', '2003-04-30',
               '2003-05-31', '2003-06-30', '2003-07-31', '2003-08-31',
               '2003-10-31', '2003-11-30',
               ...
               '2020-12-31', '2021-01-31', '2021-02-28', '2021-03-31',
               '2021-04-30', '2021-05-31', '2021-06-30', '2021-12-31',
               '2022-02-28', '2022-03-31'],
              dtype='datetime64[ns]', name='Symbol Name', length=137, freq=None)

# 변동성 팩터&기업별 변동성 생성

In [ ]:
var = pd.read_excel('C:/Users/82106/\Desktop/DART/코스피_변동성(20일).xlsx', index_col=0)
var=var.loc[inc_index,:]

In [ ]:
volatility = var[quality_index]
vol_des=volatility.describe()

In [ ]:
#기업별 변동성 
vol_comp=vol_des.loc['mean']
pd.DataFrame(vol_comp)

,mean
삼성제약,0.030873
퍼스텍,0.033522
엔에이치스팩19호,0.005700
CJ대한통운,0.021990
오리온홀딩스,0.022269
...,...
LG생활건강,0.020817
SK바이오팜,0.027114
한미사이언스,0.026686
크래프톤,0.030700


In [ ]:
# 변동성 factor 생성
vol_trans=volatility.transpose()
vol_fac=vol_trans.describe().loc['mean']
pd.DataFrame(vol_fac)

,mean
Symbol Name,
2002-08-31,0.029521
2002-10-31,0.036324
2002-11-30,0.025659
2003-04-30,0.035814
2003-05-31,0.029126
...,...
2021-05-31,0.025716
2021-06-30,0.022419
2021-12-31,0.020759


# 모멘텀 factor(시계열&횡단)

In [ ]:
income = pd.read_excel('C:/Users/82106/\Desktop/DART/코스피_수익률(1개월).xlsx', index_col=0)[quality_index]

In [ ]:
#기업별 횡단 모멘텀 생성
income=income.loc[inc_index,:]
income_cross_comp=income.iloc[136,:]
income_cross_comp

삼성제약          2.10
퍼스텍         -14.24
엔에이치스팩19호     1.69
CJ대한통운        4.44
오리온홀딩스        1.06
             ...  
LG생활건강       -9.68
SK바이오팜        6.15
한미사이언스       -1.76
크래프톤         -6.61
F&F         -16.29
Name: 2022-03-31 00:00:00, Length: 400, dtype: float64

In [ ]:
#횡단 factor 생성
income_cross=income.transpose()
income_cross_fac=income_cross.describe().loc['mean']
income_cross_fac

Symbol Name
2002-08-31     2.881496
2002-10-31     3.170890
2002-11-30     4.558613
2003-04-30    11.407017
2003-05-31     8.545672
                ...    
2021-05-31     4.044781
2021-06-30     4.784396
2021-12-31     8.608325
2022-02-28     4.625400
2022-03-31     6.100625
Name: mean, Length: 137, dtype: float64

In [ ]:
#기업별 시계열 지표 생성
income_time=(income>0).astype(int)
income_time_comp=income_time.describe().loc['mean']
income_time_comp

삼성제약         0.547445
퍼스텍          0.540146
엔에이치스팩19호    0.014599
CJ대한통운       0.620438
오리온홀딩스       0.656934
               ...   
LG생활건강       0.700730
SK바이오팜       0.058394
한미사이언스       0.583942
크래프톤         0.007299
F&F          0.029197
Name: mean, Length: 400, dtype: float64

In [ ]:
#시계열 factor 생성
income_time_fac=income_time.transpose().describe().loc['mean']
income_time_fac

Symbol Name
2002-08-31    0.3575
2002-10-31    0.3525
2002-11-30    0.3900
2003-04-30    0.5250
2003-05-31    0.4250
               ...  
2021-05-31    0.5700
2021-06-30    0.5675
2021-12-31    0.8225
2022-02-28    0.6875
2022-03-31    0.7900
Name: mean, Length: 137, dtype: float64

# 소형주 factor

In [ ]:
small = pd.read_excel('C:/Users/82106/\Desktop/DART/코스피_시가총액.xlsx', index_col=0)[quality_index]
income2 = pd.read_excel('C:/Users/82106/\Desktop/DART/코스피_수익률(6개월).xlsx', index_col=0)[quality_index]

In [ ]:
#기업별 소형주 지표
small=small.loc[inc_index,:]
income2=income2.loc[inc_index,:]
tiny=income2/small

small_comp=tiny.describe().loc['mean']
small_comp

삼성제약        -0.000029
퍼스텍          0.000065
엔에이치스팩19호   -0.000046
CJ대한통운       0.000025
오리온홀딩스       0.000010
               ...   
LG생활건강       0.000005
SK바이오팜      -0.000003
한미사이언스       0.000019
크래프톤        -0.000003
F&F          0.000004
Name: mean, Length: 400, dtype: float64

In [ ]:
#기업별 소형주 factor 생성
small_fac=tiny.transpose().describe().loc['mean']
small_fac

Symbol Name
2002-08-31   -0.000586
2002-10-31   -0.000622
2002-11-30   -0.000812
2003-04-30   -0.001042
2003-05-31   -0.000799
                ...   
2021-05-31    0.000087
2021-06-30    0.000091
2021-12-31   -0.000060
2022-02-28   -0.000063
2022-03-31   -0.000029
Name: mean, Length: 137, dtype: float64

# 다중회귀분석을 통한 가중치 찾기


In [ ]:
import statsmodels.api as sm
from sklearn import linear_model


In [ ]:
# 동일한 비율계산을 위한 표준화

def f_z_score(data): # z-score 함수 생성
    mean = np.mean(data) #평균
    std = np.std(data)   #표준편차
    z_scores = [(y-mean)/std for y in data] #z-score
    return z_scores

In [ ]:
# 각 팩터들 표준화

kospii=kospi.loc[inc_index,:]
vol_fac_z=f_z_score(vol_fac)
income_cross_fac_z=f_z_score(income_cross_fac)
income_time_fac_z=f_z_score(income_time_fac)
small_fac_z=f_z_score(small_fac)
kos_z = f_z_score(kospii.소계)

In [ ]:
kospii['vol']=vol_fac_z
kospii['소계']=kos_z
kospii['income_c']=income_cross_fac_z
kospii['income_t']=income_time_fac_z
kospii['small']=small_fac_z

In [ ]:
# 회귀분석 결과

linear_regression = linear_model.LinearRegression()
X=kospii[['vol','income_c','income_t','small']]
y=kospii['소계']
multi_model=sm.OLS(y,X)
fitted = multi_model.fit()
fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                     소계   R-squared (uncentered):                   0.372
Model:                            OLS   Adj. R-squared (uncentered):              0.353
Method:                 Least Squares   F-statistic:                              19.66
Date:                Sun, 03 Apr 2022   Prob (F-statistic):                    9.83e-13
Time:                        17:00:10   Log-Likelihood:                         -162.57
No. Observations:                 137   AIC:                                      333.1
Df Residuals:                     133   BIC:                                      344.8
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
vol            0.0756      0.081      0.938      0.350      -0.084       0.235
income_c       0.2260      0.118      1.912      0.058      -0.008       0.460
income_t       0.3949      0.108      3.641      0.000       0.180       0.609
small          0.1000      0.073      1.372      0.172      -0.044       0.244
==============================================================================
Omnibus:                       45.665   Durbin-Watson:                   1.593
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              119.567
Skew:                           1.314   Prob(JB):                     1.09e-26
Kurtosis:                       6.747   Cond. No.                         3.13
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# 표준화 시키지 않은 ver
kospiii=kospi.loc[inc_index,:]
kospiii['vol']=vol_fac
kospiii['소계']=kospi.소계
kospiii['income_c']=income_cross_fac
kospiii['income_t']=income_time_fac
kospiii['small']=small_fac

In [ ]:
kospiii

,소계,vol,income_c,income_t,small
Symbol Name,,,,,
2002-08-31,18.41,0.029521,2.881496,0.3575,-0.000586
2002-10-31,12.50,0.036324,3.170890,0.3525,-0.000622
2002-11-30,65.88,0.025659,4.558613,0.3900,-0.000812
2003-04-30,63.65,0.035814,11.407017,0.5250,-0.001042
2003-05-31,34.07,0.029126,8.545672,0.4250,-0.000799
...,...,...,...,...,...
2021-05-31,56.06,0.025716,4.044781,0.5700,0.000087
2021-06-30,92.76,0.022419,4.784396,0.5675,0.000091
2021-12-31,138.64,0.020759,8.608325,0.8225,-0.000060


In [ ]:

X2=kospiii[['vol','income_c','income_t','small']]
y2=kospiii['소계']
multi_model2=sm.OLS(y2,X2)
fitted2 = multi_model2.fit()
fitted2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                     소계   R-squared (uncentered):                   0.749
Model:                            OLS   Adj. R-squared (uncentered):              0.741
Method:                 Least Squares   F-statistic:                              99.06
Date:                Sun, 03 Apr 2022   Prob (F-statistic):                    6.59e-39
Time:                        17:00:10   Log-Likelihood:                         -708.11
No. Observations:                 137   AIC:                                      1424.
Df Residuals:                     133   BIC:                                      1436.
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
vol         -157.6145    418.433     -0.377      0.707    -985.259     670.030
income_c       3.7214      1.042      3.570      0.000       1.660       5.783
income_t     115.6350     27.884      4.147      0.000      60.482     170.788
small       1.171e+04   9879.199      1.185      0.238   -7832.195    3.12e+04
==============================================================================
Omnibus:                       51.091   Durbin-Watson:                   1.548
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              150.321
Skew:                           1.428   Prob(JB):                     2.28e-33
Kurtosis:                       7.264   Cond. No.                     1.75e+04
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.75e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
screen=pd.DataFrame(vol_comp)
screen['income_c']=income_cross_comp
screen['income_t']=income_time_comp
screen['small']=small_comp

In [ ]:
screen

,mean,income_c,income_t,small
삼성제약,0.030873,2.10,0.547445,-0.000029
퍼스텍,0.033522,-14.24,0.540146,0.000065
엔에이치스팩19호,0.005700,1.69,0.014599,-0.000046
CJ대한통운,0.021990,4.44,0.620438,0.000025
오리온홀딩스,0.022269,1.06,0.656934,0.000010
...,...,...,...,...
LG생활건강,0.020817,-9.68,0.700730,0.000005
SK바이오팜,0.027114,6.15,0.058394,-0.000003
한미사이언스,0.026686,-1.76,0.583942,0.000019
크래프톤,0.030700,-6.61,0.007299,-0.000003


In [ ]:
screen_rank=screen.fillna(0)
screen_rank.rename(columns = {'mean' : 'vol'}, inplace = True)

In [ ]:
# 팩터별 가중치 적용

screen_rank['vol_rank'] = screen_rank['vol'].rank(ascending=False)*0.5
screen_rank['income_c_rank'] = screen_rank['income_c'].rank()*3.5
screen_rank['income_t_rank'] = screen_rank['income_t'].rank()*4
screen_rank['small_rank'] = screen_rank['small'].rank()

In [ ]:
screen_rank

,vol,income_c,income_t,small,vol_rank,income_c_rank,income_t_rank,small_rank
삼성제약,0.030873,2.10,0.547445,-0.000029,40.0,427.0,822.0,38.0
퍼스텍,0.033522,-14.24,0.540146,0.000065,25.0,14.0,784.0,272.0
엔에이치스팩19호,0.005700,1.69,0.014599,-0.000046,200.0,392.0,28.0,32.0
CJ대한통운,0.021990,4.44,0.620438,0.000025,146.0,633.5,1304.0,214.0
오리온홀딩스,0.022269,1.06,0.656934,0.000010,139.5,364.0,1466.0,161.0
...,...,...,...,...,...,...,...,...
LG생활건강,0.020817,-9.68,0.700730,0.000005,164.5,35.0,1586.0,138.0
SK바이오팜,0.027114,6.15,0.058394,-0.000003,82.5,787.5,92.0,62.0
한미사이언스,0.026686,-1.76,0.583942,0.000019,88.5,199.5,1002.0,189.0
크래프톤,0.030700,-6.61,0.007299,-0.000003,42.5,56.0,8.0,63.0


In [ ]:
screen_rank['final_rank']=screen_rank['vol_rank']+screen_rank['income_c_rank']+screen_rank['income_t_rank']+screen_rank['small_rank']
screen_rank['final_ranking']=screen_rank['final_rank'].rank(ascending=False)
final_screen=screen_rank.sort_values('final_ranking')

In [ ]:
pd.DataFrame(final_screen.iloc[:10,-1])

,final_ranking
진양산업,1.0
대동,2.0
TYM,3.0
메리츠화재,4.0
한성기업,5.0
신세계 I&C,6.0
남해화학,7.0
부산산업,8.0
한솔케미칼,9.0
삼성화재,10.0


In [ ]:
final_screen.iloc[:10].index

Index(['진양산업', '대동', 'TYM', '메리츠화재', '한성기업', '신세계 I&C', '남해화학', '부산산업',
       '한솔케미칼', '삼성화재'],
      dtype='object')